In [2]:
import json, sys
import pymupdf
import json, re, sys
from pathlib import Path
import re
from openai import OpenAI
import json, pathlib
from dotenv import load_dotenv
import os
from copy import deepcopy
import os, io, json, base64, pymupdf
from openai import OpenAI
from openai import OpenAI
import json
from pathlib import Path
# from anthropic import Anthropic

In [3]:
load_dotenv()

True

In [5]:
GPTclient = OpenAI(api_key= os.getenv("OPENAI_API_KEY"))
OpenRouterclient = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("SONOMA_DUSK_ALPHA_KEY"),
)

In [ ]:
raw_toc = get_toc("books/pdfs/percolation_studies.pdf")

In [ ]:
# result  = filter_chapters_smartly(raw_toc, out_path="books/toc_json.json")

In [ ]:
def find_toc_from_outlines(PDF_PATH):
    raw_toc = get_toc(PDF_PATH)
    for x in raw_toc:
        if x['title'].lower() in ["contents", "toc", "table of contents", "brief contents"]:
            return x['page']
    return None   

In [48]:
find_toc_from_outlines("")

9

In [19]:
toc_from_llm = toc_tree_from_pdf_range("books/pdfs/percolation_studies.pdf", 9, 14, provider="openai", model="gpt-4o")

In [ ]:
# toc_from_llm

{'title': 'Table of Contents',
 'children': [{'label': '1',
   'title': 'Introduction to Percolation',
   'children': [{'label': '1.1',
     'title': 'Basic Concepts in Percolation',
     'children': []},
    {'label': '1.2', 'title': 'Percolation Probability', 'children': []},
    {'label': '1.3', 'title': 'Spanning Cluster', 'children': []},
    {'label': '1.4', 'title': 'Percolation in Small Systems', 'children': []},
    {'label': '1.5', 'title': 'Further Reading', 'children': []}]},
  {'label': '2',
   'title': 'One-Dimensional Percolation',
   'children': [{'label': '2.1',
     'title': 'Percolation Probability',
     'children': []},
    {'label': '2.2', 'title': 'Cluster Number Density', 'children': []},
    {'label': '2.3', 'title': 'Spanning Cluster', 'children': []},
    {'label': '2.4', 'title': 'Correlation Length', 'children': []}]},
  {'label': '3',
   'title': 'Infinite-Dimensional Percolation',
   'children': [{'label': '3.1',
     'title': 'Percolation Threshold',
   

In [ ]:
TOC_TITLE_CANDIDATES = {"table of contents", "contents"}
ENDNUM_RE = re.compile(r"\b\d{1,4}\s*$")

def _page_lines_with_metrics(page):
    """Return a list of dicts: {'text': str, 'bbox': (x0,y0,x1,y1), 'avg_size': float} for each visual line."""
    d = page.get_text("dict")
    lines = []
    for b in d.get("blocks", []):
        for l in b.get("lines", []):
            spans = l.get("spans", [])
            if not spans:
                continue
            text = "".join(s.get("text", "") for s in spans).strip()
            if not text:
                continue
            avg_size = sum(s.get("size", 0.0) * len(s.get("text", "")) for s in spans) / max(1, sum(len(s.get("text", "")) for s in spans))
            bbox = tuple(l.get("bbox", (0, 0, 0, 0)))
            lines.append({"text": text, "bbox": bbox, "avg_size": avg_size})
    return lines

def _looks_like_heading(line, page_height, top_fraction=0.25):
    """
    Heuristic: heading is within the top quarter of the page.
    Removed centering requirement as requested.
    """
    x0, y0, x1, y1 = line["bbox"]
    top_enough = y0 <= (page_height * top_fraction)
    return top_enough



def find_toc_start(pdf_path, max_search_ratio=0.20, min_endnum_lines=5):
    """
    Return zero-based page index of the first TOC page, or None if not found.
    Conditions:
      1) A heading line reading 'Table of Contents' or 'Contents' (case-insensitive),
         which appears as a real heading (top & size >= ~median_top*1.15).
      2) The same page has >= min_endnum_lines lines that end with an integer (page number).
    Only the first 20% of pages are scanned by default.
    """
    doc = pymupdf.open(pdf_path)
    n_pages = len(doc)
    search_upto = max(1, int(n_pages * max_search_ratio))

    for i in range(search_upto):
        page = doc[i]
        page_height = page.rect.height
        lines = _page_lines_with_metrics(page)

        # Focus on the top cluster to estimate "normal" size
        top_lines = [ln for ln in lines if ln["bbox"][1] <= page.rect.height * 0.25]
        # median_top_size = _median([ln["avg_size"] for ln in top_lines]) or _median([ln["avg_size"] for ln in lines])

        # Find a qualifying title line
        title_line = None
        for ln in top_lines[:12]:  # examine first ~12 top lines
            normalized = ln["text"].strip().lower()
            # allow punctuation/whitespace around title
            normalized = re.sub(r"[^a-z\s]", "", normalized)
            normalized = re.sub(r"\s+", " ", normalized).strip()
            if normalized in TOC_TITLE_CANDIDATES:
                # heading checks: position + size
                if _looks_like_heading(ln, page_height):
                    title_line = ln
                    break

        if not title_line:
            continue  # must have a real heading

        # Count TOC-like entries: lines ending with a number
        endnum_count = sum(1 for ln in lines if ENDNUM_RE.search(ln["text"]))
        if endnum_count >= min_endnum_lines:
            return i  # zero-based index
    print("Couldn\'t find TOC")
    return None

output =find_toc_start('books/pdfs/percolation_studies.pdf', max_search_ratio=0.20, min_endnum_lines=5)

print(output)

8


In [59]:
# pip install pymupdf openai anthropic

def toc_tree_from_pdf_range(pdf_path:str, start_page:int, end_page:int,
                            provider:str="openai", model:str="gpt-4o-mini",
                            api_key_env:str=None, pages_per_batch:int=8, dpi:int=180):
    """
    Extract a hierarchical Table of Contents tree (no page numbers) from a PDF page range using GPT/Claude vision.
    - provider: "openai" or "anthropic"
    - model: e.g., "gpt-4o-mini" or "claude-3-5-sonnet-20240620"
    Returns: dict like {"title":"Table of Contents","children":[...]}
    """

    # API client
    if api_key_env is None:
        api_key_env = "OPENAI_API_KEY" if provider.lower()=="openai" else "ANTHROPIC_API_KEY"
    api_key = os.getenv(api_key_env)
    if not api_key:
        raise RuntimeError(f"Missing {api_key_env} environment variable")
    client = OpenAI(api_key=api_key) if provider.lower()=="openai" else Anthropic(api_key=api_key)

    # PDF → base64 PNGs
    images = []
    with pymupdf.open(pdf_path) as doc:
        n = doc.page_count
        s, e = max(1, start_page), min(end_page, n)
        if s > e:
            return {"title":"Table of Contents","children":[]}
        mat = pymupdf.Matrix(dpi/72.0, dpi/72.0)
        for p in range(s-1, e):
            pix = doc[p].get_pixmap(matrix=mat, alpha=False)
            buf = io.BytesIO(pix.tobytes("png"))
            images.append(base64.b64encode(buf.getvalue()).decode("utf-8"))

    # System instructions
    SYSTEM = (
        "Extract a hierarchical outline (chapters/sections/subsections) from these Table of Contents images.\n"
        "Return ONE valid JSON object only. Schema:\n"
        '{ "title":"Table of Contents","children":[{"label":"1","title":"...","children":[{"label":"1.1","title":"...","children":[]}]}] }\n'
        "Rules: Do NOT include page numbers. Preserve order. Infer levels (1 → 1.1 → 1.1.1 …). "
        "Trim dot leaders. Merge wrapped lines. Remove headers/footers and noise."
    )

    # Helper: call LLM on a batch of images
    def call_llm(batch_b64):
        if provider.lower()=="openai":
            content = [{"type":"text","text":"Extract the outline as per the system rules."}]
            content += [{"type":"image_url","image_url":{"url":f"data:image/png;base64,{b}"}} for b in batch_b64]
            resp = client.chat.completions.create(
                model=model,
                messages=[{"role":"system","content":SYSTEM},{"role":"user","content":content}],
                response_format={"type":"json_object"}
            )
            return resp.choices[0].message.content
        else:  # anthropic
            content = [{"type":"text","text":"Extract the outline as per the system rules."}]
            content += [{"type":"image","source":{"type":"base64","media_type":"image/png","data":b}} for b in batch_b64]
            resp = client.messages.create(
                model=model,
                max_tokens=4096,
                system=SYSTEM,
                messages=[{"role":"user","content":content}]
            )
            return "".join(block.text for block in resp.content if block.type=="text")

    def parse_json(s):
        s = s.strip()
        if s.startswith("```"):
            s = s.split("```",2)[1]
        return json.loads(s)

    # Process batches
    batches = [images[i:i+pages_per_batch] for i in range(0,len(images),pages_per_batch)]
    partials = [parse_json(call_llm(b)) for b in batches if b]

    # Merge partial trees
    merged = {"title":"Table of Contents","children":[]}
    def merge_children(dst, src):
        idx = {c["label"]:i for i,c in enumerate(dst)}
        for node in src:
            lbl = node["label"]
            if lbl in idx:
                i = idx[lbl]
                if dst[i]["title"] != node["title"] and len(node["title"])>len(dst[i]["title"]):
                    dst[i]["title"] = node["title"]
                merge_children(dst[i]["children"], node.get("children", []))
            else:
                dst.append(node); idx[lbl]=len(dst)-1
    for pt in partials:
        merge_children(merged["children"], pt.get("children", []))
    return merged

In [60]:
def build_toc(PDF_PATH):
    outline_toc_st = find_toc_from_outlines(PDF_PATH)
    heuristic_toc_st = find_toc_start(PDF_PATH, max_search_ratio=0.20, min_endnum_lines=5)
    start_page = 0
    if (outline_toc_st or heuristic_toc_st):
        if not outline_toc_st:
            start_page = 1+ heuristic_toc_st
        elif not heuristic_toc_st:
            start_page = outline_toc_st
        else:
            start_page = min(outline_toc_st,1+heuristic_toc_st)
    toc_tree = toc_tree_from_pdf_range(PDF_PATH, start_page, end_page = start_page+5,
                            provider = "openai", model="gpt-4o-mini",
                            api_key_env=None, pages_per_batch=8, dpi=180)
    return toc_tree


In [61]:
build_toc('books/pdfs/percolation_studies.pdf')

{'title': 'Table of Contents',
 'children': [{'label': '1',
   'title': 'Introduction to Percolation',
   'children': [{'label': '1.1',
     'title': 'Basic Concepts in Percolation',
     'children': []},
    {'label': '1.2', 'title': 'Percolation Probability', 'children': []},
    {'label': '1.3', 'title': 'Spanning Cluster', 'children': []},
    {'label': '1.4', 'title': 'Percolation in Small Systems', 'children': []},
    {'label': '1.5', 'title': 'Further Reading', 'children': []}]},
  {'label': '2',
   'title': 'One-Dimensional Percolation',
   'children': [{'label': '2.1',
     'title': 'Percolation Probability',
     'children': []},
    {'label': '2.2', 'title': 'Cluster Number Density', 'children': []},
    {'label': '2.3', 'title': 'Spanning Cluster', 'children': []},
    {'label': '2.4', 'title': 'Correlation Length', 'children': []}]},
  {'label': '3',
   'title': 'Infinite-Dimensional Percolation',
   'children': [{'label': '3.1',
     'title': 'Percolation Threshold',
   